<a href="https://colab.research.google.com/github/mohanraju2002/MACHINE_LEARNING/blob/main/CREATE_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google. colab import files
upload=files.upload()

Saving online_shoppers_intention.csv to online_shoppers_intention.csv


In [ ]:
print("Step 1: Required librarie imported successfully")
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from imblearn.over_sampling import SMOTE

from imblearn.pipeline import Pipeline as IMBPipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)



##########################################
# Loading online_shoppers_intention.csv dataset #
##########################################
print("Step 2: Created DataFrame successfully")

df = pd.read_csv("online_shoppers_intention.csv")




###################
# Feature Engineering #
###################

print("Step 3: Feature Engineering Done successfully on Weekend,Revenue")

df['Weekend'] = df['Weekend'].replace((True, False), (1, 0))
df['Revenue'] = df['Revenue'].replace((True, False), (1, 0))

# Added Returning_Visitor column #
##############################

print("Step 4: Added Returning_Visitor column successfully")
condition = df['VisitorType']=='Returning_Visitor'
df['Returning_Visitor'] = np.where(condition, 1, 0)
df = df.drop(columns=['VisitorType'])



########################################
# Applying One Hot Encoding on Month column #
#########################################
print("Step 5: Applied one hot encoding successfully on Month column")


ordinal_encoder = OrdinalEncoder()
df['Month'] = ordinal_encoder.fit_transform(df[['Month']])



####################################
# Checking correlation on Revenue column #
#####################################

print("Step 6: Checking correlation done successfully")


result = df[df.columns[1:]].corr()['Revenue']
result1 = result.sort_values(ascending=False)

###################################
# Preparing Features as X and target as y #
###################################
print("Step 7: Preparing features as X and target as y done successfully")

X = df.drop(['Revenue'], axis=1)
y = df['Revenue']



####################################
# Preparing Train and Test Dataset#
####################################

print("Step 8: Splitting data X_train, X_test, y_train & y_test done successfully")

X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.3, random_state = 0)
print("Step 9: model_pipeline function created done successfully")
def model_pipeline(X, model):
  n_c = X.select_dtypes(exclude=['object']).columns.tolist()
  c_c = X.select_dtypes(include=['object']).columns.tolist()


  numeric_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='constant')),
  ('scaler', MinMaxScaler())
  ])

  categorical_pipeline = Pipeline([
  ('encoder', OneHotEncoder(handle_unknown='ignore'))
  ])

  preprocessor = ColumnTransformer([
  ('numeric', numeric_pipeline, n_c),
  ('categorical', categorical_pipeline, c_c)
  ], remainder='passthrough')


  final_steps = [
  ('preprocessor', preprocessor),
  ('smote', SMOTE(random_state=1)),
  ('feature_selection', SelectKBest(score_func = chi2, k =
  6)),
  ('model', model)
  ]

  return IMBPipeline(steps = final_steps)

print("Step 10: select_model function created done successfully")
def select_model(X, y, pipeline=None):
  classifiers = {}

  c_d1 = {"DummyClassifier":
  DummyClassifier(strategy='most_frequent')}
  classifiers.update(c_d1)

  c_d4 = {"RandomForestClassifier":
  RandomForestClassifier()}
  classifiers.update(c_d4)
  c_d5 = {"DecisionTreeClassifier": DecisionTreeClassifier()}
  classifiers.update(c_d5)

  c_d9 = {"KNeighborsClassifier": KNeighborsClassifier()}
  classifiers.update(c_d9)
  c_d10 = {"RidgeClassifier": RidgeClassifier()}
  classifiers.update(c_d10)
  c_d14 = {"SVC": SVC()}
  classifiers.update(c_d14)
  mlpc = {
  "MLPClassifier (paper)":
  MLPClassifier(hidden_layer_sizes=(27, 50),
  max_iter=300,
  activation='relu',
  solver='adam',
  random_state=1)
  }

  c_d16 = mlpc
  classifiers.update(c_d16)

  cols = ['model', 'run_time', 'roc_auc']
  df_models = pd.DataFrame(columns = cols)

  for key in classifiers:
    start_time = time.time()
    print()
    print("Step 12: model_pipeline run successfully on",  key)
    pipeline = model_pipeline(X_train, classifiers[key])

    cv = cross_val_score(pipeline, X, y, cv=10,
    scoring='roc_auc')
    row = {'model': key,
    'run_time': format(round((time.time() -
    start_time)/60,2)),
    'roc_auc': cv.mean(),
    }

    df_models = pd.concat([df_models,
    pd.DataFrame([row])], ignore_index=True)

    df_models = df_models.sort_values(by='roc_auc',
    ascending = False)
  return df_models

#################################
# Access Model select_model function #
#################################
print("Step 11: Accessing select_model function done successfully")

models = select_model(X_train, y_train)



#############################
# Let’s see total model with score #
#############################
print("Step 13: Accessing select_model function done successfully")
print(models)

Step 1: Required librarie imported successfully
Step 2: Created DataFrame successfully
Step 3: Feature Engineering Done successfully on Weekend,Revenue
Step 4: Added Returning_Visitor column successfully
Step 5: Applied one hot encoding successfully on Month column
Step 6: Checking correlation done successfully
Step 7: Preparing features as X and target as y done successfully
Step 8: Splitting data X_train, X_test, y_train & y_test done successfully
Step 9: model_pipeline function created done successfully
Step 10: select_model function created done successfully
Step 11: Accessing select_model function done successfully

Step 12: model_pipeline run successfully on DummyClassifier


/tmp/ipython-input-2558673358.py:70: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Weekend'] = df['Weekend'].replace((True, False), (1, 0))
/tmp/ipython-input-2558673358.py:71: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Revenue'] = df['Revenue'].replace((True, False), (1, 0))
/tmp/ipython-input-2558673358.py:202: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To


Step 12: model_pipeline run successfully on RandomForestClassifier

Step 12: model_pipeline run successfully on DecisionTreeClassifier

Step 12: model_pipeline run successfully on KNeighborsClassifier

Step 12: model_pipeline run successfully on RidgeClassifier

Step 12: model_pipeline run successfully on SVC

Step 12: model_pipeline run successfully on MLPClassifier (paper)
Step 13: Accessing select_model function done successfully
                    model run_time   roc_auc
6   MLPClassifier (paper)     1.92  0.903247
0                     SVC      1.0  0.889927
1  RandomForestClassifier     0.41  0.884583
2         RidgeClassifier     0.01  0.856459
3    KNeighborsClassifier     0.01  0.841013
4  DecisionTreeClassifier     0.02  0.736005
5         DummyClassifier     0.01  0.500000
